### Proyecto práctico

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _Space Invaders_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, una solución óptima será alcanzada cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

   1) Implementar la red neuronal que se usará en la solución
    
   2) Implementar las distintas piezas de la solución DQN
    
   3) Justificar la respuesta en relación a los resultados obtenidos

IMPORTANTE:

- Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.

- Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).

- Necesitaréis instalar `gymnasium[atari,accept-rom-license]`


#### Importar librerías

In [37]:
# from PIL import Image
# import numpy as np
# #import gymnasium as gym

# import matplotlib.pyplot as plt
# import torch

#### Configuración base

In [38]:
# INPUT_SHAPE = (84, 84)
# WINDOW_LENGTH = 4

# env_name = 'SpaceInvaders-v4'
# env = gym.make(env_name)

# np.random.seed(123)
# nb_actions = env.action_space.n
# state, info = env.reset()

In [39]:
# state.shape

# 1) Implementación de la red neuronal

In [7]:
# if IN_COLAB:
#   %pip install gym==0.17.3
#   %pip install git+https://github.com/Kojoley/atari-py.git
#   %pip install keras-rl2==1.0.5
#   %pip install tensorflow==2.8
# else:
#   %pip install gym==0.17.3
#   %pip install git+https://github.com/Kojoley/atari-py.git
#   %pip install pyglet==1.5.0
#   %pip install h5py==3.1.0
#   %pip install Pillow==9.5.0
#   %pip install keras-rl2==1.0.5
#   %pip install Keras==2.2.4
#   %pip install tensorflow==2.5.3
#   %pip install torch==2.0.1
#   %pip install agents==1.4.0

In [1]:
import numpy as np

import sys

import tensorflow.keras
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Tensor Flow Version: 2.5.3
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [2]:
from __future__ import division

from PIL import Image

import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
import numpy as np

In [3]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

In [4]:
# In this example, we need to preprocess the observations
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [5]:
import gym
import numpy as np
env_name = 'SpaceInvaders-v4'
env = gym.make(env_name)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [6]:
print("Numero de acciones disponibles: " + str(nb_actions))
print("\n")
print("Formato de las observaciones:")
env.observation_space

Numero de acciones disponibles: 6


Formato de las observaciones:


Box(0, 255, (210, 160, 3), uint8)

In [7]:
# Next, we build our model. We use the same model that was described by Mnih et al. (2015).
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
print(K.image_data_format())
if K.image_data_format() == 'channels_last':
    # (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    # (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')
# se podria sustituir el stride alto por capas de pooling


model.add(Convolution2D(filters=32, kernel_size= (8,8), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Convolution2D(filters= 64, kernel_size=(4,4), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(filters= 128, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

channels_last
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 84, 84, 32)        8224      
_________________________________________________________________
activation (Activation)      (None, 84, 84, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 64)        32832     
_________________________________________________________________
activation_1 (Activation)    (None, 42, 42, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 42

# 2) Implementación de la solución DQN

In [8]:
memory = SequentialMemory(limit=1_000_000,             # memoria muy grande 
                          window_length=WINDOW_LENGTH) # 4
processor = AtariProcessor()                           # creamos la instancia de preprocesamiento

In [9]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),  # definimos la policy
                              attr='eps',          # pero con un scheduler
                              value_max=1.,        # empiezaa en 1
                              value_min=.05,       # termina en 0.1 en el ultimo step
                              value_test=.05,      # en testeo será de 0.05
                              nb_steps=1_800_000)  #

In [10]:
dqn = DQNAgent(model=model, 
               nb_actions=nb_actions, 
               policy=policy,
               memory=memory, 
               processor=processor,
               nb_steps_warmup=20_000,       # tenemos experience replay
               gamma=.99,                    # el discount reward
               target_model_update=10_000,   # cada 5000 itereaciones se actualiza se actualiza el modelo
               train_interval=4)             # cada cuantas iteraciones se hace un step de entrenaminto

dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

In [12]:
# Training part
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50_000)]

callbacks += [FileLogger(log_filename, interval=50_000)]

In [30]:
dqn.fit(env, 
        callbacks=callbacks, 
        nb_steps=2_000_000, 
        log_interval=10_000, 
        visualize=False)

Training for 2000000 steps ...
Interval 1 (0 steps performed)


c:\Users\plane\miniconda3\envs\venv_RL\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10000/10000 [==============================] - 63s 6ms/step - reward: 0.0124
15 episodes - episode_reward: 8.200 [1.000, 21.000] - ale.lives: 2.160

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 63s 6ms/step - reward: 0.0134
15 episodes - episode_reward: 8.467 [2.000, 14.000] - ale.lives: 2.126

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 345s 34ms/step - reward: 0.0157
14 episodes - episode_reward: 11.500 [3.000, 26.000] - loss: 0.007 - mae: 0.027 - mean_q: 0.040 - mean_eps: 0.987 - ale.lives: 2.084

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 343s 34ms/step - reward: 0.0145
12 episodes - episode_reward: 11.833 [4.000, 23.000] - loss: 0.007 - mae: 0.060 - mean_q: 0.083 - mean_eps: 0.982 - ale.lives: 2.054

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 342s 34ms/step - reward: 0.0121
13 episodes - episode_reward: 9.769 [4.000, 26.000] - los

In [13]:
dqn.save_weights(weights_filename, overwrite=True)

### Evaluación del modelo resultante

In [20]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_SpaceInvaders-v4_weights_2000000.h5f'   # .format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 19.000, steps: 926
Episode 2: reward: 31.000, steps: 1284
Episode 3: reward: 30.000, steps: 1165
Episode 4: reward: 20.000, steps: 976
Episode 5: reward: 17.000, steps: 746
Episode 6: reward: 29.000, steps: 1261
Episode 7: reward: 29.000, steps: 1309
Episode 8: reward: 22.000, steps: 1122
Episode 9: reward: 23.000, steps: 939
Episode 10: reward: 18.000, steps: 858


# 3) Justificación de los parámetros seleccionados y de los resultados obtenidos

En esta implementación de DQN se utilizan la biblioteca de keras-rl y gym en vez de gymnasium y pytorch.

## Parametros de observación y procesado del entorno

- Las dimensiones de entrada son de 84x84 y la ventana es de 4 frames, lo que indica que se toman 4 fotogramas consecutivos como entrada de la red 
- Clase atariprocessor: convierte las imagenes del juego en un formato adecuado para la red neuronal, lo que incluye redimensionar la imagen a 84x84, convertirla a escala de grises, se normaliza el lote de imágene,s y se limita o clipea la recompensa a un rango de -1 a 1.
- Se define el entorno  'SpaceInvaders-v4' de OpenAI Gym.
- El modelo consta de 6 acciones.

## Parámetros del modelo 

Se construye un modelo secuencial con varias capas convolucionales y de pooling, seguidas de capas densas.

- Capas Convolucionales y de Pooling: reducimos la dimensionalidad mientras aumentamos el numero de filtros.
- Funcion de activación ReLU
- La capa de salida tiene tantas neuronas como acciones disponibles y utiliza una activación lineal.

## Parámetros de la policy y memoria

Se utiliza una política epsilon greedy mediante la técnica simulated annealing: 

- empieza con un epsilón de 1.0, es decir el proceso de entrenamiento es completamente aleatorio ( fase de exploración ). 
- Disminuye linealmente hasta 0.05 en 1_800_000 mil steps.
- En la fase de test también se utiliza un epsilón de 0.05 para mantener un margen de aleatoriedad o "magia".
- Se establece un límite o buffer de memoria de 1,000,000, lo que significa que el agente puede almacenar hasta un millón de experiencias pasadas.

## Configuración del agente

- nb_steps_warmup: 20,000 pasos de "calentamiento" antes de que comience el entrenamiento activo, es decir, no se actualiza el modelo en este periodo.
- Gamma (γ): Un discount factor de 0.99.
- target_model_update: 10,000 pasos para la actualización del modelo objetivo.
- train_interval: El agente realiza una acción de entrenamiento cada 4 pasos.

## Entrenamiento

- optimizador adam con un learning rate de 0.00025 
- callbacks para guardar modelos intermendios cada 50 mil pasos
- 2_000_000 de pasos para el entrenamiento en total. El entrenamiento a durado más de 25 horas.
- La fase de explotación con un epsilón de 0.05 dura 

## Resultados

La fase de testo muestra cierta inconsistencia (resultados dispersos), sin embargo, en la mayoría de casos llega sobradamente a los 20 puntos de media, cosa que no ocurria en los ensayos previos donde se ha entrenado el modelo con medio millón y después con un millón de steps.
